In [ ]:
#https://github.com/markoarnauto/biterm
import pandas as pd
import numpy as np
import os
import re
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import scipy
import nltk
nltk.download('stopwords')
!pip install biterm
!pip install pyLDAvis
import pyLDAvis
from biterm.btm import oBTM 
from sklearn.feature_extraction.text import CountVectorizer
from biterm.utility import vec_to_biterms, topic_summuary 
nltk.download('punkt') # needed for apply function 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 3.7 MB/s 
  Created wheel for biterm: filename=biterm-0.1.5-cp37-cp37m-linux_x86_64.whl size=195882 sha256=3876e69dc2b5269e3cc90d8f445dde96896c3eac3a71f500cf288b12354546dd
  Stored in directory: /root/.cache/pip/wheels/35/45/73/c4a79327f13728b195c12f35aa9331f897e37786f8e446cb09
Successfully built biterm
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 9.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=a0b64223da69aeef2ff785c52e404196c6d46b3c520d21bd4dc4c192bb6fac16
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.getcwd() # to see working directory
os.chdir("/content/drive/MyDrive/Web 3.0") #changing workig directory to google drive

In [ ]:
df=pd.read_excel("Downloaded_9-12_Web3 _7 days.xlsx")
#df.columns # to check number of column
df_originalt = df[df["Engagement Type"].isnull()] # Only looking at original tweets 
df_fulltext =df_originalt[["Full Text"]] # Then selecting the original texts 


In [ ]:
def remove_content(text):
    text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', text, flags=re.MULTILINE) # To Remove http, https and the other normal url type special characters
    text=re.sub(r'\S+\.com\S+','',text) #remove urls."" mean no space or nothing
    text=re.sub(r'\@\w+','',text) #remove mentions
    text =re.sub(r'\#\w+','',text) #remove hashtags
    text =re.sub(r"^RT\s+",'',text) #remove rt's. re.sub() used to replace substrings; Syntax: re.sub(pattern,replacement,string) & will replace the matches in string with repl.
    return text

In [ ]:
from nltk.stem.snowball import stopwords
def process_text(text, stem= False): #clean text
    text=remove_content(text)
    text = re.sub('[^A-Za-z0-9]', ' ', text.lower()) #remove non-alphabets(?); lower is method applied to string text; matching each word in text-replacing with lower version
    tokenized_text = word_tokenize(text) #tokenize
    clean_text = [
         word for word in tokenized_text
         #if word not in stopwords.word("english")
         if word not in stopwords.words("english")
    ]
    if stem:
        stemmer = PorterStemmer() # initializig a poterstemmer object
        clean_text=[stemmer.stem(word) for word in clean_text]
    return ' '.join(clean_text) 

# String.join(iterable).The join() method takes all items in an iterable and joins them into one string.
# Example: let myTuple = ("John", "Peter", "Vicky"), x = "#".join(myTuple), print(x),  John#Peter#Vicky

In [ ]:
df_fulltext["Full Text"]=df_fulltext["Full Text"].astype("str") # In the initial dataset not all the values were string. So this convert every value to string (shown as object type)
df_reduced = df_fulltext.sample(frac=1) # for testing the analytical pipeline, use smaller sample size 
df_reduced['cleaned_tweets']=df_reduced["Full Text"].apply(lambda x: process_text(x))
df_reduced["tokenized_cleantweets"] = df_reduced["cleaned_tweets"].apply(lambda x:word_tokenize(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
#pyLDAvis has issues with documents with very few tokens. So here we are filtering for tweets with atleast 3 tokens in them.
i=0
pylda = []
while i<len(df_reduced ):
  #print(i)
  if len (df_reduced.iloc[i,2]) > 5:
    A = df_reduced.iloc[i,2]
  else:
    A= 0
  pylda.append(A)
  i+=1
  pylda = [i for i in pylda if i!=0]
df_a = pd.DataFrame()
df_a ["Clean"] = pylda # this is a df containing full text with token >3. Look for method to reduce creation of df's

In [ ]:
df_a["Clean"]=df_a["Clean"].astype("str") # str is a req for vectorizer
#docs_r = df_r['cleaned_tweets'].tolist() # Converting inviduals tweets from excel to list
docs_r = df_a["Clean"].tolist()

In [ ]:
#vec = CountVectorizer(stop_words='english')
#X = vec.fit_transform(docs_r).toarray()
vec = CountVectorizer(stop_words="english")
X = vec.fit_transform(docs_r).toarray() # Sparse Matrix which is being converted to array. So going from lean to dense.

In [ ]:
vocab = np.array(vec.get_feature_names())
biterms = vec_to_biterms(X)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
import multiprocessing as mp
mp.cpu_count() # to see how many cores I have

2

In [ ]:
btm = oBTM(num_topics=5, V=vocab)
for i in range(0, len(biterms), 100): 
  biterms_chunk = biterms[i:i + 100]
  btm.fit(biterms_chunk, iterations=10)
topics = btm.transform(biterms)

100%|██████████| 10/10 [00:35<00:00,  3.53s/it]


In [ ]:
vis = pyLDAvis.prepare(btm.phi_wz.T, topics, np.count_nonzero(X, axis=1), vocab, np.sum(X, axis=0))
pyLDAvis.save_html(vis,'simple_btm.html')

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [ ]:
topic_summuary(btm.phi_wz.T, X, vocab, 10)

Topic 0 | Coherence=-130.81 | Top words= web3 web nft like join people check crypto new community
Topic 1 | Coherence=-135.41 | Top words= web3 web nft time space nfts blockchain people today metaverse
Topic 2 | Coherence=-139.04 | Top words= web3 web blockchain new community join crypto space metaverse users
Topic 3 | Coherence=-138.15 | Top words= web3 web 2022 million nft game games funding platform metaverse
Topic 4 | Coherence=-123.11 | Top words= web3 new world community digital nft web nfts join connect


{'coherence': [-130.80897775921983,
  -135.40775126892092,
  -139.03812562818015,
  -138.15190510432294,
  -123.10528524598128],
 'top_words': [array(['web3', 'web', 'nft', 'like', 'join', 'people', 'check', 'crypto',
         'new', 'community'], dtype='<U132'),
  array(['web3', 'web', 'nft', 'time', 'space', 'nfts', 'blockchain',
         'people', 'today', 'metaverse'], dtype='<U132'),
  array(['web3', 'web', 'blockchain', 'new', 'community', 'join', 'crypto',
         'space', 'metaverse', 'users'], dtype='<U132'),
  array(['web3', 'web', '2022', 'million', 'nft', 'game', 'games',
         'funding', 'platform', 'metaverse'], dtype='<U132'),
  array(['web3', 'new', 'world', 'community', 'digital', 'nft', 'web',
         'nfts', 'join', 'connect'], dtype='<U132')]}